## Análisis de tipos de producto CvLAC

In [1]:
import pandas as pd
import requests
import time
import json
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str,
'COD_RH_CREA':str
 }

# Check some products

In [2]:
pf=pd.read_json('product_ids_2022.json',dtype=dtype)

In [3]:
pf.shape

(390012, 2)

In [4]:
pf=pf.drop_duplicates().reset_index(drop=True)
pf.shape

(388375, 2)

## Articles

In [5]:
import getpass
RUN=True
if RUN:
    APIKEY=getpass.getpass('APIKEY: ')

APIKEY:  ········


In [7]:
ds=pf.loc[7]
TEST=True#False
if TEST:
    url=f'http://clustercien.udea.edu.co:8080/scienti/product?apikey={APIKEY}&model_year=2022&COD_RH={ds.COD_RH}&COD_PRODUCTO={ds.COD_PRODUCTO}'
    r=requests.get(url)
    d=r.json()
    try:
        d['NIVEL_MAX']=d.get('product_type')[0].get('COD_TIPO_PRODUCTO')+': '+d.get('product_type')[0].get('TXT_NME_TIPO_PRODUCTO')
    except:
        d['NIVEL_MAX']='-1: UNKNOWN'
d['NIVEL_MAX']        

'111: Publicado en revista especializada'

In [ ]:
#%%writefile comment_this_line_for_run.py
store=1000
l=[]
for i,ds in pf.iterrows():
    print(i,end='\r')
    try:
        r=requests.get(f'http://clustercien.udea.edu.co:8080/scienti/product?apikey={APIKEY}&model_year=2021&COD_RH={ds.COD_RH}&COD_PRODUCTO={ds.COD_PRODUCTO}')
    except:
        time.sleep(0.1)
        continue

    d=r.json()
    try:
        d['NIVEL_MAX']=d.get('product_type')[0].get('COD_TIPO_PRODUCTO')+': '+d.get('product_type')[0].get('TXT_NME_TIPO_PRODUCTO')
    except:
        d['NIVEL_MAX']='-1: UNKNOWN'
        
    if d['NIVEL_MAX']!='111: Publicado en revista especializada':
        continue
    #Articles with DOI, group or SGL_CATEGORY
    if d.get('TXT_DOI') or d.get('group') or d.get('SGL_CATEGORY'):
        l.append(d)
        if i%store==0:
            print(i,end='\r')
            f=open('product_type_111_2022.json','w')
            json.dump(l,f)
            f.close()            
    time.sleep(0.01)
f=open('product_type_111_2022.json','w')
json.dump(l,f)
f.close()    

In [28]:
len(l)

1

### Load articles

In [6]:
import json

In [24]:
f=open('product_type_111_2021.json','r')
l=json.load(f)
f.close()

Only articles with declared group

In [29]:
l=[x for x in l if x.get('group')]

In [28]:
f=open('product_type_111_group_2021.json','w')
json.dump(l,f)
f.close()

In [7]:
f=open('product_type_111_group_2021.json','r')
l=json.load(f)
f.close()

In [8]:
l=pd.DataFrame(l)

In [9]:
from IPython.display import JSON

In [10]:
#JSON(l.iloc[3].to_dict())

Prepare requested fields:

Título, año, volumen, página inicial, página final, autor, ISSN, nombre revista, doi, url.

In [11]:
for k in ['TXT_VOLUMEN_REVISTA','TXT_PAGINA_INICIAL','TXT_PAGINA_FINAL']:
    l[k]=l.get('details').str[0].str.get('article').str[0].str.get(k)

In [12]:
for k in ['TXT_ISSN_SEP','TXT_NME_REVISTA']:
    l[k]=l.get('details').str[0].str.get('article').str[0
                   ].str.get('journal').str[0].str.get(k)

In [13]:
l.get('author_others').apply(lambda l:
                            [d.get('TXT_TOTAL_NAMES') for d in l if d.get('TXT_TOTAL_NAMES')]
                                if isinstance(l,list) else []
                        ).apply(len).max()

61

In [14]:
l['autores']=l.get('author_others').apply(lambda l:
                            [d.get('TXT_TOTAL_NAMES') for d in l if d.get('TXT_TOTAL_NAMES')]
                                if isinstance(l,list) else []
                        ).apply(lambda l: '; '.join(l))

In [15]:
for k in ['NRO_ID_GRUPO','COD_ID_GRUPO','NME_GRUPO']:
    l[k]=l.get('group').str[0].str.get(k)

In [16]:
c=set(l.columns).difference([
'COD_RH',
'COD_PRODUCTO',
'TXT_DOI',
'TXT_NME_PROD', 
'TXT_NME_REVISTA', 
'TXT_VOLUMEN_REVISTA',   
'TXT_PAGINA_INICIAL', 
'TXT_PAGINA_FINAL',
'NRO_ANO_PRESENTA',
'TXT_DOI', 
'TXT_ISSN_SEP', 
'TXT_WEB_PRODUCTO',
'NRO_ID_GRUPO',
'COD_ID_GRUPO', 
'NME_GRUPO',
'CAT_MED',
'SGL_CATEGORIA',
'autores'
])

In [17]:
l=l.drop(c,axis='columns')

In [18]:
l[:2]

,COD_RH,COD_PRODUCTO,TXT_NME_PROD,NRO_ANO_PRESENTA,CAT_MED,SGL_CATEGORIA,NRO_ID_GRUPO,TXT_WEB_PRODUCTO,TXT_DOI,TXT_VOLUMEN_REVISTA,TXT_PAGINA_INICIAL,TXT_PAGINA_FINAL,TXT_ISSN_SEP,TXT_NME_REVISTA,autores,COD_ID_GRUPO,NME_GRUPO
0,0000000132,42,Proteomic and biological characterization of t...,2011,ART-ART_A1,ART-ART_A1,00000000008117,NaN,NaN,75,655,667,1874-3919,Journal of Proteomics,Jessica Paola Rey Suarez; Vitelbina Nuñez Rang...,COL0014476,"Toxinología, Alternativas Terapéuticas y Alime..."
1,0000000132,43,Snake venomics of the pit vipers Porthidium na...,2012,ART-ART_A1,ART-ART_A1,00000000008117,NaN,NaN,75,1675,1689,1874-3919,Journal of Proteomics,Jessica Paola Rey Suarez; Tsai WC; Juan Jose ...,COL0014476,"Toxinología, Alternativas Terapéuticas y Alime..."


In [19]:
l.shape

(31981, 17)

In [20]:
l.drop_duplicates(subset=['TXT_NME_PROD'] ).shape

(28586, 17)

### Normalize DOIS

In [21]:
import re
def normalize_dois(s):
    ss=s #store NaN
    if isinstance(s,str):
        ss=s.strip().lower()
        ss=re.sub('\.$','',ss) #proper doi end
        ss=re.split('https*:\/\/.*doi\.org\/',ss)[-1].strip() #also works with oadoi!
        #Quality checks
        if (re.search('https*:\/\/',ss) #exclude Improper URL
            or ss.find('/')==-1 # Must contains at least one '\/', exclude: '-', '--','00','n.a.', etc.
            or re.search('^.\/.$',ss) #exclude 'N/A', 'n/a' etc
            ): 
            ss=None
        else:
            ss=ss.split(':')[-1].strip() #Proper doi split
            ss=re.split('^.*doi\.org\/',ss)[-1].strip() #also works with oadoi!
    return ss

assert normalize_dois('10.18273/RevMed')=='10.18273/revmed'
assert normalize_dois('doi:10.18273/revmed')=='10.18273/revmed'
assert normalize_dois('doi: 10.18273/RevMed')=='10.18273/revmed'
assert normalize_dois('10.3390/molecules23102662.')=='10.3390/molecules23102662'
assert normalize_dois('https://doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://dx.doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://oadoi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('dx.doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://www.udea.edu.co/10.1155/2017/1702506') is None
assert normalize_dois('N/A') is None
assert normalize_dois('-') is None
assert normalize_dois('') is None
assert isinstance(normalize_dois(float('NaN')),float) #return NaN
assert normalize_dois(['10.18273/revmed','10.3823/1406'])==['10.18273/revmed','10.3823/1406'] #ignore not str objects

In [22]:
l['TXT_DOI']=l['TXT_DOI'].apply(normalize_dois)

In [23]:
l.to_excel('product_type_2021_111_Publicado_en_revista_especializada_grupos.xlsx',index=False,engine='xlsxwriter')